In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)
print(mnist.train.num_examples)
print(mnist.validation.num_examples)
print(mnist.test.num_examples)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
55000
5000
10000


In [5]:
# parameter and hyperparameter setting
pixel=28
image_size=pixel*pixel
n_class=10
n_hidden1=500
n_hidden2=500
n_latent=20
batch_size=100
training_epochs=5
itr=int(mnist.train.num_examples*training_epochs/batch_size)
print(itr)

lr=0.001

display_step=1
example_to_show=10

2750


In [6]:
x=tf.placeholder('float32',[None,image_size])
keep_prob=tf.placeholder('float32')
z_in=tf.placeholder('float32',[None,n_latent])

In [7]:
def weight_variable(shape):
#     init=tf.Variable(tf.random_normal(shape=shape,dtype='float32'))
    init=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init)

def bias_variable(shape):
#     init=tf.Variable(tf.random_normal(shape=shape,dtype='float32'))
    init=tf.constant(0.1,shape=shape)
    return tf.Variable(init)

def fc_layer(inputs,n_outputs,activation_fn=tf.nn.relu,keep_prob=1):
    n_inputs=int(inputs.shape[1])
    W_fc=weight_variable([n_inputs,n_outputs])
    b_fc=bias_variable([n_outputs])
    logit_fc=tf.matmul(inputs,W_fc)+b_fc
    _a_fc=activation_fn(logit_fc)
    a_fc=tf.nn.dropout(_a_fc,keep_prob=keep_prob)
    return a_fc,logit_fc

def encoder(inputs,n_hidden1,n_hidden2,n_latent,keep_prob=1):
    fc_layer1,_=fc_layer(inputs,n_hidden1,activation_fn=tf.nn.sigmoid,keep_prob=keep_prob)
    fc_layer2,_=fc_layer(fc_layer1,n_hidden2,activation_fn=tf.nn.sigmoid,keep_prob=keep_prob)
    
    w_encoder=weight_variable([n_hidden2,n_latent*2])
    b_encoder=bias_variable([n_latent*2])
    gaussian_params=tf.matmul(fc_layer2,w_encoder)+b_encoder
    
    mean=gaussian_params[:,:n_latent]
    # to make stddev be positive, add softplus 
    # to make stddev be stable, add a small epsilon
    stddev=1e-6+tf.nn.softplus(gaussian_params[:,n_latent:])
    return mean,stddev

def decoder(z,n_hidden2,n_hidden1,n_outputs,keep_prob=1):
    fc_layer1,_=fc_layer(z,n_hidden2,activation_fn=tf.nn.sigmoid,keep_prob=keep_prob)
    fc_layer2,_=fc_layer(fc_layer1,n_hidden1,activation_fn=tf.nn.sigmoid,keep_prob=keep_prob)
    
    w_decoder=weight_variable([n_hidden1,n_outputs])
    b_decoder=bias_variable([n_outputs])
    outputs=tf.sigmoid(tf.matmul(fc_layer2,w_decoder)+b_decoder)
    
    return outputs

def variational_autoencoder(inputs,n_hidden1,n_hidden2,n_latent,keep_prob=1):
    # encoder
    mu,std=encoder(inputs,n_hidden1,n_hidden2,n_latent,keep_prob)
    
    # sampling by re-parameterization technique
    latent_variable=mu+std*tf.random_normal(tf.shape(mu),mean=0,stddev=1,dtype='float32')
    
    # decoder
    n_outputs=int(inputs.shape[1])
    x_=decoder(latent_variable,n_hidden2,n_hidden1,n_outputs,keep_prob)
    x_=tf.clip_by_value(x_,1e-8,1-1e-8)
    
    # loss
    marginal_likelihood=tf.reduce_sum(inputs*tf.log(x_)+(1-inputs)*tf.log(1-x_),1)
    KL_divergence=0.5*tf.reduce_sum(tf.square(mu)+tf.square(std)-tf.log(1e-7+tf.square(std))-1,1)
    
    marginal_likelihood=tf.reduce_mean(marginal_likelihood)
    KL_divergence=tf.reduce_mean(KL_divergence)
    
    ELBO=marginal_likelihood-KL_divergence
    
    loss=-ELBO
    
    return x_,latent_variable,loss,-marginal_likelihood,KL_divergence,mu,std

def generator(z,n_hidden2,n_hidden1,n_outputs):
    return decoder(z,n_hidden2,n_hidden1,n_outputs,keep_prob=1)



In [8]:
x_,z,loss,neg_marginal_likelihood,KL_divergence,mu,std=variational_autoencoder(
    x,n_hidden1,n_hidden2,n_latent,keep_prob=keep_prob)
optimizer=tf.train.AdamOptimizer().minimize(loss)

In [9]:
sess=tf.Session() 
init=tf.global_variables_initializer()
sess.run(init,feed_dict={keep_prob:0.9})
training_epochs=10
mnist.train.next_batch(100)
for itrr in range(itr):
    batch_xs,batch_ys=mnist.train.next_batch(100)

    _,tot_loss,loss_likelihood,loss_divergence=sess.run(
        (optimizer,loss,neg_marginal_likelihood,KL_divergence),
    feed_dict={x:batch_xs,keep_prob:0.9})
    if itrr%100==0:
        print("itreation: %d: L_tot %03.2f L_likelihood %03.2f L_divergence %03.2f" % (
            itrr+1,tot_loss,loss_likelihood,loss_divergence))


itreation: 0: L_tot 727.12 L_likelihood 693.45 L_divergence 33.67
itreation: 100: L_tot 216.00 L_likelihood 215.59 L_divergence 0.41
itreation: 200: L_tot 212.66 L_likelihood 211.88 L_divergence 0.78
itreation: 300: L_tot 191.13 L_likelihood 187.77 L_divergence 3.37
itreation: 400: L_tot 193.21 L_likelihood 189.90 L_divergence 3.30
itreation: 500: L_tot 184.03 L_likelihood 179.63 L_divergence 4.40
itreation: 600: L_tot 175.89 L_likelihood 170.72 L_divergence 5.17
itreation: 700: L_tot 174.96 L_likelihood 169.51 L_divergence 5.45
itreation: 800: L_tot 172.89 L_likelihood 167.24 L_divergence 5.65
itreation: 900: L_tot 177.47 L_likelihood 169.97 L_divergence 7.50
itreation: 1000: L_tot 157.73 L_likelihood 149.45 L_divergence 8.28
itreation: 1100: L_tot 154.21 L_likelihood 145.10 L_divergence 9.11
itreation: 1200: L_tot 155.52 L_likelihood 146.01 L_divergence 9.51
itreation: 1300: L_tot 145.64 L_likelihood 136.39 L_divergence 9.25
itreation: 1400: L_tot 140.58 L_likelihood 131.24 L_diverge

In [ ]:
sess.close()
